In [1]:
####################################################
#  
#  This will read from an application authorized box file
#  Files may be accessed programmatically  from Box to an application
#  IBM Guidance: https://ibm.ent.box.com/v/Dev4Box
#
#  1. Create a new box app: https://ibm.ent.box.com/developers/console
#     - Custom App
#     - Server Authentication with JWT
#     - OAuth 2.0 Credentials <- Don't Change
#     - Application Access = Application
#     - Client id - take note of this value for later
#     - Application Scopes - check only "Read and write all files and folder stored in box"
#     - Advanced Features - Both toggled off
#     - Generate a public private key pair and save the resulting json file for later (JWTAuth.from_settings_file)
#     - Save application 
#     - General Setting Tab - click "review and submit"
#   2. Request application permission: https://w3.ibm.com/#/support/article/box_development
#   3. Once approved, use the code below to programmatically  determine the user service account info
#      - Determine the service account login - client.user().get().login
#      - Use the service account login - invite it to a box file that has the files you want it to be able to access
#   4. Programatically determine the folder and file ids to access the data
#
#  RESOURCES:
#   https://ibm.ent.box.com/v/Dev4Box
#   https://w3.ibm.com/#/support/article/box_development
#   http://opensource.box.com/box-python-sdk/tutorials/intro.html
#   https://support.box.com/hc/en-us/community/posts/360049197133-Problem-following-API-example
#   https://box-python-sdk.readthedocs.io/en/stable/boxsdk.object.html#module-boxsdk.object.folder
#
##########################

In [3]:
%%capture
!pip install BoxSDK
!pip install "boxsdk[jwt]==2.6.1"

In [15]:
################################################
# AFTER THE BOX APP HAS BEEN AUTHORIZED,
# USE THIS CODE TO OBTAIN SERVICE ACCOUNT LOGIN
# 
#  3. Once approved, use the code below to programatically determine the user service account info
#
################################################

from boxsdk import JWTAuth, Client

config = JWTAuth.from_settings_file('PATH_TO_BOX_config.json')
client = Client(config)
service_account = client.user().get()
# print('Service Account user ID is {0}'.format(service_account.id))
# print('Service Account user name is {0}'.format(service_account.name))
login=client.user().get().login          #<-user to invite to share folder/file in box
print('INVITE THIS SERVICE ACCOUNT LOGIN TO SHARE BOX FOLDER OR BOX FILE: ', login) 
root_folder = client.folder(folder_id='0').get()

In [16]:
############################################################################
# ACCESS TO BOX
#
#    4. Programatically determine the folder and file ids to access the data
#
############################################################################

def fetch_folder_id_from_box(foldername):
    if foldername=='0':
        return '0'
    else:
        #locate box folder
        FOLDER_ID=''
        folder_items = root_folder.get_items(limit=100,offset=0, sort='date',direction='DESC')#get newest
        for folder_item in folder_items:
            if folder_item.name.startswith(foldername):
                #print('Folder Name: ', folder_item.name)
                FOLDER_ID = folder_item.id
                #print('Folder Id:   ', FOLDER_ID)
                break
        return FOLDER_ID

def fetch_file_id_from_box(foldername,filename):
#   #locate box folder
    FOLDER_ID= fetch_folder_id_from_box(foldername)
    #locate box file in folder
    FILE_ID=''   
    FILES_FOLDER = client.folder(folder_id=FOLDER_ID).get()   
    file_items = FILES_FOLDER.get_items(limit=100,offset=0, sort='date', direction='DESC')
    for file_item in file_items:
        if file_item.name.startswith(filename):
            #print('File Name: %s/%s' %(root_folder.name, file_item.name))
            FILE_ID = file_item.id
            #print('File Id:  ', FILE_ID)
            break
    return FILE_ID

def box_file_exists(foldername,filename):
    FILE_EXISTS=False
    #locate box folder
    FOLDER_ID=fetch_folder_id_from_box(foldername)
    FILE_ID=''   
    FILES_FOLDER = client.folder(folder_id=FOLDER_ID).get()   
    file_items = FILES_FOLDER.get_items(limit=100,offset=0, sort='date', direction='DESC')
    for file_item in file_items:
        if file_item.name.startswith(filename):
            FILE_EXISTS=True
            break
    return FILE_EXISTS

def upload_file_to_box(boxfoldername,boxfilename): 
    if box_file_exists(boxfoldername,boxfilename):
        box_file_id=fetch_file_id_from_box(boxfoldername,boxfilename)
    #     updated_file = client.file(file_id).update_contents(file_path)
        updated_file = client.file(box_file_id).update_contents(boxfilename)
        print('File "{0}" has been updated'.format(updated_file.name))    
    else: #file does not exist, upload new file
        #get box folderid using box folder name
        box_folder_id=fetch_folder_from_box(boxfoldername)
        new_file = client.folder(box_folder_id).upload(boxfilename)
        print('File "{0}" uploaded to Box with file ID {1}'.format(new_file.name, new_file.id)) 


In [ ]:
##################################
#  USAGE EXAMPLE
##################################
folder_in_box=''   # if root folder (file is not in a folder), then '0', else the name of the shared folder in box
file_in_box=''     # name of shared file in box
local_file_name='' # file to write box file contents to locally

BOX_FILE_ID=fetch_file_id_from_box(folder_in_box,file_in_box)
BOX_FILE=client.file(BOX_FILE_ID)
BOX_FILE_CONTENTS=client.file(BOX_FILE_ID).content().decode("utf-8")

open(local_file_name, 'a').write(BOX_FILE_CONTENTS)
print(open(local_file_name, 'r').read())